# InThon Submission Code

## Imports


In [ ]:
!pip install -q git+https://github.com/huggingface/peft.git transformers bitsandbytes datasets
!pip install git+https://github.com/salaniz/pycocoevalcap

import torch
from PIL import Image
import requests
from transformers import Blip2Processor, Blip2ForConditionalGeneration, InstructBlipProcessor, InstructBlipForConditionalGeneration, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from torch.utils.data import DataLoader

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/salaniz/pycocoevalcap to /tmp/pip-req-build-29qvkppl
  Running command git clone --filter=blob:none --quiet https://github.com/salaniz/pycocoevalcap /tmp/pip-req-build-29qvkppl
  Resolved https://github.com/salaniz/pycocoevalcap to commit a24f74c408c918f1f4ec34e9514bc8a76ce41ffd
  Preparing metadata (setup.py) ... done


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"사용 중인 디바이스: {device}")

사용 중인 디바이스: cuda


## Load Model & Processor


In [ ]:
bnb_config = BitsAndBytesConfig(
    # load_in_8bit=True,
    load_in_4bit=True,
    # bnb_4bit_use_double_quant=True,
    # bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

processor = InstructBlipProcessor.from_pretrained("Salesforce/instructblip-vicuna-13b")
model = InstructBlipForConditionalGeneration.from_pretrained(
    "Salesforce/instructblip-vicuna-13b",
    quantization_config=bnb_config,
    # device_map="auto"
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - 

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
for name, module in model.named_modules():
    print(name)


vision_model
vision_model.embeddings
vision_model.embeddings.patch_embedding
vision_model.encoder
vision_model.encoder.layers
vision_model.encoder.layers.0
vision_model.encoder.layers.0.self_attn
vision_model.encoder.layers.0.self_attn.dropout
vision_model.encoder.layers.0.self_attn.qkv
vision_model.encoder.layers.0.self_attn.projection
vision_model.encoder.layers.0.layer_norm1
vision_model.encoder.layers.0.mlp
vision_model.encoder.layers.0.mlp.activation_fn
vision_model.encoder.layers.0.mlp.fc1
vision_model.encoder.layers.0.mlp.fc2
vision_model.encoder.layers.0.layer_norm2
vision_model.encoder.layers.1
vision_model.encoder.layers.1.self_attn
vision_model.encoder.layers.1.self_attn.dropout
vision_model.encoder.layers.1.self_attn.qkv
vision_model.encoder.layers.1.self_attn.projection
vision_model.encoder.layers.1.layer_norm1
vision_model.encoder.layers.1.mlp
vision_model.encoder.layers.1.mlp.activation_fn
vision_model.encoder.layers.1.mlp.fc1
vision_model.encoder.layers.1.mlp.fc2
visio

## Checking Trainable Parameters
> This code section must not be modified! Any modifications may affect your evaluation. This section directly relates to the number of trainable parameters, which is one of the grading criteria. If any false reporting regarding this section is detected, it may result in penalties during evaluation.

In [ ]:
def disable_trainable_parameters(model):
    for param in model.parameters():
        param.requires_grad = False  # 모든 파라미터를 비학습 상태로 설정
    model.eval()  # 모델을 평가 모드로 전환

disable_trainable_parameters(model)

In [ ]:
def print_trainable_params(model):
    trainable_params = 0
    all_params = 0
    for _, param in model.named_parameters():
        all_params += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(f"전체 파라미터 수: {all_params / 1e6:.2f}M")
    print(f"학습 가능한 파라미터 수: {trainable_params}")
    print(f"파라미터 비율: {100 * trainable_params / all_params:.2f}%")

print_trainable_params(model)

전체 파라미터 수: 7272.78M
학습 가능한 파라미터 수: 0
파라미터 비율: 0.00%


## Load Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import pandas as pd
from torch.utils.data import Dataset, DataLoader, random_split
import random

dataset_path = os.path.join('/content/drive/MyDrive/Inthon/content', 'dataset.csv')
data_df = pd.read_csv(dataset_path)

train_df = data_df[data_df['train'] == True]
val_df = data_df[data_df['val'] == True]
test_df = data_df[data_df['test'] == True]
print(f"Training set size: {len(train_df)}, Validation set size: {len(val_df)}, Test set size: {len(test_df)}")

num_epochs = 0
print(f"Total training data point size: {len(train_df) * num_epochs}")

Training set size: 3013, Validation set size: 200, Test set size: 200
Total training data point size: 0


## Define Custom Dataset


In [ ]:
from torch.utils.data import Dataset
from PIL import Image
import requests
from io import BytesIO

class CustomImageCaptionDataset(Dataset):
    def __init__(self, df, processor, test=False):
        self.df = df
        self.processor = processor
        self.test = test

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if not self.test:
            image_url = self.df.iloc[idx]["url"]
            image_id = self.df.iloc[idx]["Image_ID"]
            caption = self.df.iloc[idx]["Paragraph"]

            response = requests.get(image_url)
            image = Image.open(BytesIO(response.content)).convert('RGB')

            inputs = self.processor(
                images=image,
                text=caption,
                padding="max_length",
                truncation=True,
                max_length=256,
                return_tensors="pt"
            )
            inputs = {k: v.squeeze(0) for k, v in inputs.items()}
            inputs['labels'] = inputs['input_ids'].clone()

            inputs['image_url'] = image_url
            inputs['Image_ID'] = image_id
            inputs['reference_caption'] = caption

        else:
            image_url = self.df.iloc[idx]["url"]
            image_id = self.df.iloc[idx]["Image_ID"]

            inputs = dict()
            inputs['image_url'] = image_url
            inputs['Image_ID'] = image_id

        return inputs

In [ ]:
train_dataset = CustomImageCaptionDataset(train_df, processor)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=16)

val_dataset = CustomImageCaptionDataset(val_df, processor)
val_dataloader = DataLoader(val_dataset, shuffle=False, batch_size=8)

test_dataset = CustomImageCaptionDataset(test_df, processor, test=True)
test_dataloader = DataLoader(test_dataset, shuffle=False, batch_size=8)

## Evaluation and Metric Calculation
We calculate SPICE, CLIPScore, and CHAIRf metrics on the test dataset. \\


**SPICE**: A score that evaluates text quality by measuring semantic content based on objects, relationships, and attributes present in image captions. \\

**CLIPScore**: A metric that measures the similarity between images and text using the CLIP model. \\

**CHAIR**: A metric that evaluates how well captions align with images, focusing on examining which objects are included in the captions. Unlike CHAIRi and CHAIRs which are more precision-oriented, CHAIRf considers both precision and recall aspects in its evaluation. \\


SPICE: https://arxiv.org/abs/1607.08822 \\
CLIPScore: https://arxiv.org/abs/2104.08718 \\
CHAIR: https://arxiv.org/abs/1809.02156 \\

In [ ]:
from transformers import CLIPProcessor, CLIPModel
import torch
from tqdm import tqdm
import requests
from PIL import Image
from io import BytesIO
from pycocoevalcap.spice.spice import Spice
from collections import Counter
import spacy

evaluation_objects = [
    "man", "woman", "tree", "sky", "building", "window", "shirt", "wall",
    "sign", "grass", "water", "table", "train", "plate", "car", "dog", "cat",
    "giraffe", "light", "pole", "plane", "boy", "zebra", "bus", "elephant",
    "ground", "hair", "girl", "horse", "cloud", "hand", "clock", "people",
    "snow", "bird", "chair", "fence", "glass", "floor", "bear", "boat",
    "street", "head", "door", "road", "shoe", "leg", "eye", "hat"
]
spacy.prefer_gpu()

nlp = spacy.load("en_core_web_sm")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
spice_scorer = Spice()

def get_singular_form(word):
    return word.lemma_

def extract_objects_from_caption(caption, object_list):
    doc = nlp(caption)
    objects_in_caption = set()

    for token in doc:
        word = token.lemma_.lower()
        if word in object_list:
            objects_in_caption.add(word)

    return objects_in_caption

def calculate_chair_metrics(generated_caption, image_objects):
    caption_objects = extract_objects_from_caption(generated_caption, evaluation_objects)

    hallucinated_objects = caption_objects - set(image_objects)
    missing_objects = set(image_objects) - caption_objects
    true_positives = caption_objects & set(image_objects)

    precision = len(true_positives) / (len(true_positives) + len(hallucinated_objects)) if len(caption_objects) > 0 else 0
    recall = len(true_positives) / (len(true_positives) + len(missing_objects)) if len(image_objects) > 0 else 0

    chairf = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    print(f"환각 객체: {hallucinated_objects}, 누락 객체: {missing_objects}, 캡션 객체: {caption_objects}, 이미지 객체: {image_objects}")

    return chairf

def calculate_metrics(image, generated_caption, reference_caption, image_objects):
    spice_score, _ = spice_scorer.compute_score({0: [reference_caption]}, {0: [generated_caption]})

    inputs = clip_processor(text=generated_caption,
                            images=image,
                            return_tensors="pt",
                            padding="max_length",
                            truncation=True,
                            max_length=77).to(device)
    outputs = clip_model(**inputs)
    logits_per_image = outputs.logits_per_image
    clip_score = logits_per_image.item()

    chairf = calculate_chair_metrics(generated_caption, image_objects)
    print(f'spice_score: {spice_score}, clip_score: {clip_score}, chairf: {chairf}')

    return spice_score, clip_score, chairf

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
from tqdm import tqdm
import matplotlib.pyplot as plt

val_results = []

with torch.no_grad():
    for batch in tqdm(val_dataloader, desc="Calculating SPICE, CLIPScore, and CHAIRf"):
        image_urls = batch['image_url']
        image_ids = batch['Image_ID']
        reference_captions = batch['reference_caption']

        images = []
        for image_url in image_urls:
            response = requests.get(image_url)
            image = Image.open(BytesIO(response.content)).convert('RGB')
            images.append(image)

        image_objects_batch = [
            extract_objects_from_caption(ref_caption, evaluation_objects)
            for ref_caption in reference_captions
        ]

        inputs = processor(images=images, text=['Describe the background of the image, describe every object in the image and its appearance in detail.'] * len(images), return_tensors="pt").to(device)

        generated_ids = model.generate(
            **inputs,
            do_sample=True,
            num_beams=4,
            max_length=160,
            min_length=100,
            top_p=0.92,
            repetition_penalty=1.2,
            length_penalty=1.1,
            temperature=0.7
        )

        generated_captions = processor.batch_decode(generated_ids, skip_special_tokens=True)
        generated_captions = [caption.strip() for caption in generated_captions]

        print(generated_captions)

        existing_obj = [[] for _ in range(len(generated_captions))]

        for i, caption in enumerate(generated_captions):
            for obj in evaluation_objects:
                if ' ' + obj + ' ' in caption.lower() or ' ' + obj + '.' in caption.lower() or ' ' + obj + ',' in caption.lower():
                    existing_obj[i].append(obj)


        prompts = ["Simply describe following objects in the image: " + ", ".join(obj_list) + ". Focus on their appearance, color, actions, and interactions with surrounding objects."
                    for obj_list in existing_obj]

        print(existing_obj)

        inputs = processor(images=images, text=prompts, return_tensors="pt", padding=True).to(device)

        generated_ids = model.generate(
            **inputs,
            do_sample=True,
            num_beams=4,
            max_length=180,
            min_length=120,
            top_p=0.92,
            repetition_penalty=1.2,
            length_penalty=1.1,
            temperature=0.7
        )

        generated_captions = processor.batch_decode(generated_ids, skip_special_tokens=True)
        generated_captions = [caption.strip() for caption in generated_captions]


        for image_id, generated_caption, reference_caption, image, image_objects in zip(
            image_ids, generated_captions, reference_captions, images, image_objects_batch
        ):
            spice_score, clip_score, chairf = calculate_metrics(
                image, generated_caption, reference_caption, image_objects
            )

            plt.imshow(image)
            plt.axis('off')
            plt.show()

            print(f"Generated Caption: {generated_caption}")
            print(f"Reference Caption: {reference_caption}")

            val_results.append({
                "Image_ID": image_id,
                "generated_caption": generated_caption,
                "reference_caption": reference_caption,
                "spice_score": spice_score,
                "clip_score": clip_score,
                "chairf": chairf,
            })


        results_df = pd.DataFrame(val_results)

        average_spice = results_df["spice_score"].mean()
        average_clip_score = results_df["clip_score"].mean()
        average_chairf = results_df["chairf"].mean()

        print(f"Average SPICE Score: {average_spice:.4f}")
        print(f"Average CLIPScore: {average_clip_score:.4f}")
        print(f"Average CHAIRf Score: {average_chairf:.4f}")

        def calculate_custom_score(spice_score, clip_score, chairf):
            custom_score = (0.4 * spice_score) + (0.2 * (clip_score / 250)) + (0.4 * chairf)
            return custom_score

        results_df["custom_score"] = results_df.apply(
            lambda row: calculate_custom_score(
                    row["spice_score"], row["clip_score"], row["chairf"]
            ), axis=1
        )

        average_custom_score = results_df["custom_score"].mean()
        print(f"Average Custom Score: {average_custom_score:.4f}")

Calculating SPICE, CLIPScore, and CHAIRf:   0%|          | 0/25 [00:04<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
results_df = pd.DataFrame(val_results)

average_spice = results_df["spice_score"].mean()
average_clip_score = results_df["clip_score"].mean()
average_chairf = results_df["chairf"].mean()

print(f"Average SPICE Score: {average_spice:.4f}")
print(f"Average CLIPScore: {average_clip_score:.4f}")
print(f"Average CHAIRf Score: {average_chairf:.4f}")

def calculate_custom_score(spice_score, clip_score, chairf):
    custom_score = (0.4 * spice_score) + (0.2 * (clip_score / 250)) + (0.4 * chairf)
    return custom_score

results_df["custom_score"] = results_df.apply(
    lambda row: calculate_custom_score(
        row["spice_score"], row["clip_score"], row["chairf"]
    ), axis=1
)

average_custom_score = results_df["custom_score"].mean()
print(f"Average Custom Score: {average_custom_score:.4f}")

## Save Results

In [ ]:
submission_data = []

with torch.no_grad():
    for batch in tqdm(test_dataloader, desc="Generating Captions"):
        image_urls = batch['image_url']
        image_ids = batch['Image_ID']

        images = []
        for image_url in image_urls:
            response = requests.get(image_url)
            image = Image.open(BytesIO(response.content)).convert('RGB')
            images.append(image)

        inputs = processor(images=images, text=['Describe the background of the image, describe every object in the image and its appearance in detail.'] * len(images), return_tensors="pt").to(device)
        generated_ids = model.generate(
            **inputs,
            do_sample=True,
            num_beams=4,
            max_length=160,
            min_length=100,
            top_p=0.92,
            repetition_penalty=1.2,
            length_penalty=1.1,
            temperature=0.7
        )

        generated_captions = processor.batch_decode(generated_ids, skip_special_tokens=True)
        generated_captions = [caption.strip() for caption in generated_captions]

        print(generated_captions)

        existing_obj = [[] for _ in range(len(generated_captions))]

        for i, caption in enumerate(generated_captions):
            for obj in evaluation_objects:
                if ' ' + obj + ' ' in caption.lower() or ' ' + obj + '.' in caption.lower() or ' ' + obj + ',' in caption.lower():
                    existing_obj[i].append(obj)

        print(existing_obj)

        prompts = ["Simply describe following objects in the image: " + ", ".join(obj_list) + ". Focus on their appearance, color, actions, and interactions with surrounding objects."
                    for obj_list in existing_obj]

        inputs = processor(images=images, text=prompts, return_tensors="pt", padding=True).to(device)
        generated_ids = model.generate(
            **inputs,
            do_sample=True,
            num_beams=4,
            max_length=180,
            min_length=120,
            top_p=0.92,
            repetition_penalty=1.2,
            length_penalty=1.1,
            temperature=0.7
        )

        generated_captions = processor.batch_decode(generated_ids, skip_special_tokens=True)
        generated_captions = [caption.strip() for caption in generated_captions]

        print()
        print(generated_captions)

        for image_id, generated_caption in zip(
            image_ids, generated_captions
        ):
            submission_data.append({
                "Image_ID": image_id,
                "generated_caption": generated_caption
            })


Generating Captions:   0%|          | 0/25 [00:00<?, ?it/s]

['In the background of the image, there is a train station with a red train on the tracks. The train appears to be stopped at the station, possibly waiting for passengers to board or disembark. The train station is surrounded by trees and other greenery, providing a scenic backdrop for the train. Additionally, there are several people standing around the train station, likely waiting for their train or observing the scene.', 'In the image, a man is standing in front of a brick fireplace. He is holding a video game controller and appears to be playing a bowling video game on a TV mounted above the fireplace. The TV is positioned high up on the wall, making it easy for the man to see the game while standing in front of the fireplace. There are also a few books scattered around the room, adding to the cozy atmosphere of the scene.', 'In the background of the image, there is a green building with a sign that reads "Irish Independent Delicatessen." There is also a traffic light in front of 

Generating Captions:   4%|▍         | 1/25 [01:03<25:16, 63.20s/it]


['a red train and a group of people standing on the train platform, waiting for the train to arrive. The train is in the process of pulling into the station, with people standing on the platform waiting for the train to arrive. The train is a red color, and the people are standing on the platform, waiting for the train to arrive. The train is a red color, and the people are standing on the platform, waiting for the train to arrive. The train is a red color, and the people are standing on the platform, waiting for the train to arrive. The train is a red color, and the people are standing on the platform, waiting for the train to arrive. The train is a red color, and the', "A man in a blue shirt is standing in front of a brick wall, holding a white bowling ball. The man is playing a game of bowling on a large screen television, which is mounted on the wall in front of him. The television is displaying the game of bowling, and the man is holding the bowling ball as if he is about to thro

Generating Captions:   4%|▍         | 1/25 [01:08<27:29, 68.73s/it]


KeyboardInterrupt: 

In [ ]:
submission_df = pd.DataFrame(submission_data)
submission_df.to_csv("submission_team4_13.csv", index=False)

print("Submission file 'submission.csv' created successfully.")